# Jobs dashboard project
Scraping indeed.com for web developer jobs in Singapore to provide an almost-real-time (once a day) summary of the demand for tech jobs in Singapore. It also includes details on job requirements (e.g. skills, years of experience, programming languages, etc).

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv
import time
import json
from pandas.io.json import json_normalize

In [2]:
%matplotlib inline
#import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import plotly.tools as tls
from pandas import set_option
set_option("display.max_rows", 10)

## Let's scrape indeed.com!
link: http://www.indeed.com.sg/jobs?q=developer&l=Singapore

In [ ]:
#The standard beautifulsoup incantation to scrape a given URL
url = ('http://www.indeed.com.sg/jobs?q=developer&l=Singapore')
r = requests.get(url)
soup = BeautifulSoup(r.text, "html")
print soup.prettify()

In [ ]:
#Creating an array containing job titles
jobTitles = []
for item in soup.find_all('a',{'data-tn-element':'jobTitle'}):
    jobTitles.append(item.attrs['title'])
print jobTitles

In [ ]:
#Creating an array containing corresponding companies
companies = []
for item in soup.find_all('span',{'class':'company'}):
    companies.append(item.text.replace('\n', '').replace('  ', ''))
companies

## Putting it altogether

In [23]:
pages = pd.Series(np.arange(0,1000,50))
consolidated_table = [] # let's create an array to hold the arrays returned from each scraped page
for page in pages:
    try: 
        #The standard beautifulsoup incantation to scrape a given URL

        url = ('http://www.indeed.com.sg/jobs?q=developer&l=Singapore&limit=50&radius=10&start=' + ''' + i ''')
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html")

        #Creating an array containing job titles
        jobTitles = []
        for item in soup.find_all('a',{'data-tn-element':'jobTitle'}):
            jobTitles.append(item.attrs['title'])

        #Creating an array containing corresponding companies
        companies = []
        for item in soup.find_all('span',{'class':'company'}):
            companies.append(item.text.replace('\n', '').replace('  ', ''))

        #Creating an array containing corresponding job descriptions
        descriptions = []
        for item in soup.find_all('span',{'class':'summary'}):
            descriptions.append(item.text.replace('\n', ''))
            
        #concatenating the table
        table = pd.DataFrame({'jobTitle': pd.Series(jobTitles),
                             'company': pd.Series(companies),
                             'description': pd.Series(descriptions)})
        table = table[['jobTitle', 'description', 'company']]

        consolidated_table.append(table)

    except:
        pass
consolidated_table = pd.concat(consolidated_table, axis = 0)

,jobTitle,description,company
0,Software Engineer (.Net),Our client a service provider within the healt...,Berkley Recruitment Group
1,WEB PROGRAMMER,"A web programmer plans, creates and codes on t...",Emboss Creative & Design Pte Ltd
2,Software Programmer,• Minimum 4 years of professional experience w...,INFO-TECH SYSTEMS INTEGRATORS PTE LTD
3,Senior IT Developer,Snr IT Developer with J2EE Development & Deplo...,Aryan Search Pte Ltd
4,Web Developer,Job Responsibilities: * Design and develop ap...,Infinity JobCom
...,...,...,...
50,"Assistant Manager/Manager, Data Analytics",Programming knowledge and experience with Hado...,SourceClear
51,Software Engineer - Agent,As a Java Engineer in the agents and developer...,Paktor
52,Senior Android Lead/ Developer,"The projects on our plate are amazing, and we'...",1LIVEWELL PTE. LTD.
53,Web Developer / Programmer,Requirements: Minimum Diploma in Computer Scie...,GoDigital Singapore


In [8]:
consolidated_table.to_csv("jobs.csv",  encoding='utf-8')